<a href="https://colab.research.google.com/github/itta0607/WIP/blob/main/wip_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Dropout, GlobalAveragePooling1D, Attention
from tensorflow.keras.callbacks import EarlyStopping

# Drive内のCSVフォルダへのパス
train_csv_path = Path("/content/drive/MyDrive/gym_keypoints/train")
test_csv_path = Path("/content/drive/MyDrive/gym_keypoints/test")


In [ ]:
def calc_angle(a, b, c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    return np.arccos(np.clip(cosine_angle, -1.0, 1.0))

def extract_features_from_csv_folder(csv_dir, max_frames=300):
    X, y = [], []
    for label_dir in csv_dir.iterdir():
        if not label_dir.is_dir(): continue
        label = label_dir.name
        for csv_file in label_dir.glob("*.csv"):
            df = pd.read_csv(csv_file)
            coords = df.select_dtypes(include=[np.number]).dropna(axis=1).values
            coords = coords.reshape(coords.shape[0], -1, 3)  # [T, 33, 3]

            angles = []
            for i in range(coords.shape[0]):
                row = []
                row.append(calc_angle(coords[i,11], coords[i,13], coords[i,15]))  # right elbow
                row.append(calc_angle(coords[i,12], coords[i,14], coords[i,16]))  # left elbow
                row.append(calc_angle(coords[i,5], coords[i,11], coords[i,13]))   # right shoulder
                row.append(calc_angle(coords[i,6], coords[i,12], coords[i,14]))   # left shoulder
                angles.append(row)
            angles = np.array(angles)

            vel = np.diff(angles, axis=0, prepend=angles[0:1])
            combined = np.concatenate([angles, vel], axis=1)

            # padding
            if combined.shape[0] < max_frames:
                pad_len = max_frames - combined.shape[0]
                combined = np.pad(combined, ((0, pad_len), (0,0)), mode='constant')
            else:
                combined = combined[:max_frames]

            X.append(combined)
            y.append(label)
    return np.array(X), np.array(y)


In [ ]:
X_train, y_train_raw = extract_features_from_csv_folder(train_csv_path)
X_test, y_test_raw = extract_features_from_csv_folder(test_csv_path)

# ラベルの数値化 & ワンホット
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_raw)
y_test_encoded = le.transform(y_test_raw)

y_train = to_categorical(y_train_encoded)
y_test = to_categorical(y_test_encoded)


In [ ]:
# shape: (samples, time_steps, features)
n_samples, n_time, n_feat = X_train.shape

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, n_feat)).reshape(n_samples, n_time, n_feat)
X_test_scaled = scaler.transform(X_test.reshape(-1, n_feat)).reshape(X_test.shape[0], n_time, n_feat)


In [ ]:
def build_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = Attention()([x, x])  # Self-attention
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model(X_train_scaled.shape[1:], num_classes=y_train.shape[1])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 8)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 300, 128)  │     37,376 │ input_layer[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 300, 128)  │          0 │ bidirectional[0]… │
│ (Attention)         │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 13)        │        845 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 46,477 (181.55 KB)

 Trainable params: 46,477 (181.55 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# クラス重み
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
cw_dict = dict(zip(np.unique(y_train_encoded), class_weights))

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


In [ ]:
model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=30,
    batch_size=8,
    callbacks=[early_stop],
    class_weight=cw_dict
)


Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 409ms/step - accuracy: 0.1077 - loss: 2.5726 - val_accuracy: 0.3077 - val_loss: 2.4401
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 247ms/step - accuracy: 0.4832 - loss: 2.3477 - val_accuracy: 0.4615 - val_loss: 2.2998
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - accuracy: 0.7054 - loss: 2.0862 - val_accuracy: 0.5000 - val_loss: 2.1429
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.6619 - loss: 1.7859 - val_accuracy: 0.4808 - val_loss: 1.9638
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 203ms/step - accuracy: 0.5944 - loss: 1.5994 - val_accuracy: 0.5385 - val_loss: 1.8812
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 287ms/step - accuracy: 0.6326 - loss: 1.3335 - val_accuracy: 0.5962 - val_loss: 1.8718
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 324ms/step - accuracy: 0.6494 - loss: 1.0885 - val_accuracy: 0.5577 - val_loss: 1.8594
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 205ms/step - accuracy: 0.7419 - loss: 0.9126 - val_accuracy: 0.5962 - val_loss:

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("✅ Accuracy:", accuracy_score(y_true, y_pred))
print("✅ F1-score (macro):", f1_score(y_true, y_pred, average='macro'))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred, target_names=le.classes_))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 425ms/step
✅ Accuracy: 0.6538461538461539
✅ F1-score (macro): 0.651903651903652

📊 Classification Report:
                 precision    recall  f1-score   support

      arm_curl       1.00      0.50      0.67         4
   bench_press       0.67      1.00      0.80         4
 bent_over_row       0.67      1.00      0.80         4
   chest_press       1.00      0.50      0.67         4
  dumbbell_fly       1.00      1.00      1.00         4
   hammer_curl       1.00      0.50      0.67         4
 lat_pull_down       1.00      1.00      1.00         4
 lateral_raise       0.29      0.50      0.36         4
 leg_extension       1.00      0.50      0.67         4
     leg_press       0.33      0.50      0.40         4
       pec_fly       0.00      0.00      0.00         4
shoulder_press       0.40      0.50      0.44         4
         squat       1.00      1.00      1.00         4

      accuracy                           0.65        52
     macro avg       0.